In [1]:
# Import libraries

In [2]:
import kfp
import kfp.components as comp
import kfp.dsl as dsl
import requests

from kfp.components import InputPath, OutputPath, create_component_from_func
from typing import NamedTuple

In [3]:
# Download Dataset, Preprocess Data, Load Data

In [4]:
def download_data_IMDB(load_data_path: comp.OutputPath(str())):
    
    import os
    import shutil
    import tensorflow as tf

    from pathlib import Path
    
    """
    ## Set all paths
    """
    
    dataset_dir = os.path.join(load_data_path, 'aclImdb')
    train_dir = os.path.join(dataset_dir, 'train')
    test_dir = os.path.join(dataset_dir, 'test')
    
    if not os.path.exists(load_data_path):
        os.makedirs(load_data_path) 
    
    """
    ## Load IMDB Data
    """
    
    dataset_file = os.path.join(load_data_path, 'aclImdb_v1.tar.gz')

    if not os.path.exists(dataset_file):

        url = "https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz"

        dataset = tf.keras.utils.get_file("aclImdb_v1", url,
                                            untar=True, cache_dir=load_data_path,
                                            cache_subdir='')

        """
        ## Remove unused directory
        """

        remove_dir = os.path.join(train_dir, 'unsup')

        if os.path.isdir(remove_dir):
            shutil.rmtree(remove_dir)
                
   
    

In [5]:
download_IMDB_op = kfp.components.create_component_from_func(download_data_IMDB,
                                                             output_component_file='load_data_component.yaml',
                                                             base_image="python:3.8",
                                                             packages_to_install=['tensorflow', 'pathlib'])

In [6]:
def download_data_FP(load_data_path: comp.OutputPath(str())):
    
    import os
    import pandas as pd

    from pathlib import Path
    from datasets import load_dataset, DownloadMode
    
    """
    ## Set all paths
    """
    
    dataset_dir = os.path.join(load_data_path, 'aclImdb')
    train_dir = os.path.join(dataset_dir, 'train')
    train_pos_dir = os.path.join(train_dir, 'pos')
    train_neg_dir = os.path.join(train_dir, 'neg')
    test_dir = os.path.join(dataset_dir, 'test')
    test_pos_dir = os.path.join(test_dir, 'pos')
    test_neg_dir = os.path.join(test_dir, 'neg')
    
    dirs = [load_data_path, dataset_dir, train_dir, train_pos_dir, train_neg_dir, test_dir, test_pos_dir, test_neg_dir]
    
    for directory in dirs:
        if not os.path.exists(directory):
            os.makedirs(directory) 
                
    """
    ## Load Financial Phrasebank Data
    """
    
    def write_to_folder():
        dataset = pd.read_csv(dataset_file)

        pos_df = dataset[dataset['label'] == 2]
        write_txt(pos_df, train_pos_dir, test_pos_dir)

        neg_df = dataset[dataset['label'] == 0]
        write_txt(neg_df, train_neg_dir, test_neg_dir)
        
    def write_txt(dataset, train_dir, test_dir):
        i = 0

        for row in dataset.values:
            if i % 2 == 1:
                filename = os.path.join(train_dir, 'FP' + str(i) + '.txt')
                f = open(filename, 'w', encoding='utf-8')
                f.write(row[0])
                f.close()
                i += 1
            elif i % 2 == 0:
                filename = os.path.join(test_dir, 'FP' + str(i) + '.txt')
                f = open(filename, 'w', encoding='utf-8')
                f.write(row[0])
                f.close()
                i += 1     
    
    dataset_file = os.path.join(load_data_path, 'financial_phrasebank.csv')
    
    if not os.path.exists(dataset_file):
        dataset = load_dataset(path='financial_phrasebank',
                               name='sentences_allagree',
                               download_mode=DownloadMode.FORCE_REDOWNLOAD)

        for split, data in dataset.items():
            data.to_csv(dataset_file, index=None)
        
        write_to_folder()

In [7]:
download_FP_op = kfp.components.create_component_from_func(download_data_FP,
                                                            output_component_file='load_data_component.yaml',
                                                            base_image="python:3.8",
                                                            packages_to_install=['pathlib', 'datasets', 'pandas'])

In [8]:
# Preprocessing Data

In [107]:
def preprocess_data(batch_size: int,
                    load_data_path_IMDB: comp.InputPath(str()),
                    load_data_path_FP: comp.InputPath(str()),
                    preprocess_data_path: comp.OutputPath(str())):
    
    import tensorflow as tf
    import shutil
    import os 
    import stat
    
    # Merge Datasets

    def copytree(src, dst, symlinks=False, ignore=None):
        if not os.path.exists(dst):
            os.makedirs(dst)
            shutil.copystat(src, dst)
        lst = os.listdir(src)
        if ignore:
            excl = ignore(src, lst)
            lst = [x for x in lst if x not in excl]
        for item in lst:
            s = os.path.join(src, item)
            d = os.path.join(dst, item)
            if symlinks and os.path.islink(s):
                if os.path.lexists(d):
                    os.remove(d)
                os.symlink(os.readlink(s), d)
                try:
                    st = os.lstat(s)
                    mode = stat.S_IMODE(st.st_mode)
                    os.lchmod(d, mode)
                except:
                    pass  # lchmod not available
            elif os.path.isdir(s):
                copytree(s, d, symlinks, ignore)
            else:
                shutil.copy2(s, d)


    os.makedirs(preprocess_data_path, exist_ok = True)            
                
    copytree(load_data_path_IMDB, preprocess_data_path)  
    copytree(load_data_path_FP, preprocess_data_path) 
    
    dataset_dir = os.path.join(preprocess_data_path, 'aclImdb')
    train_dir = os.path.join(dataset_dir, 'train')
    test_dir = os.path.join(dataset_dir, 'test') 
    
    # Split Datasets
    raw_train_ds = tf.keras.preprocessing.text_dataset_from_directory(
                    train_dir,
                    batch_size=batch_size,
                    validation_split=0.2,
                    subset="training",
                    seed=1337
    )
    
    raw_val_ds = tf.keras.preprocessing.text_dataset_from_directory(
                    train_dir,
                    batch_size=batch_size,
                    validation_split=0.2,
                    subset="validation",
                    seed=1337
    )
    
    raw_test_ds = tf.keras.preprocessing.text_dataset_from_directory(
                     test_dir, batch_size=batch_size
    )
    
    tf.data.Dataset.save(raw_train_ds, f'{preprocess_data_path}/raw_train')
    tf.data.Dataset.save(raw_val_ds, f'{preprocess_data_path}/raw_val')
    tf.data.Dataset.save(raw_test_ds, f'{preprocess_data_path}/raw_test')

In [108]:
preprocess_op = kfp.components.create_component_from_func(preprocess_data,
                                                          output_component_file='preprocess_data_component.yaml',
                                                          base_image="python:3.8",
                                                          packages_to_install=['tensorflow'])

In [109]:
# Vectorize Data

In [110]:
def vectorize_data(preprocess_data_path: comp.InputPath(str()),
                   vectorize_data_path: comp.OutputPath(str())):
    
    import os
    import pickle
    import tensorflow as tf
    
    #@title Choose a BERT model to fine-tune

    bert_model_name = 'bert_en_uncased_L-12_H-768_A-12'  #@param ["bert_en_uncased_L-12_H-768_A-12", "bert_en_cased_L-12_H-768_A-12", "bert_multi_cased_L-12_H-768_A-12", "small_bert/bert_en_uncased_L-2_H-128_A-2", "small_bert/bert_en_uncased_L-2_H-256_A-4", "small_bert/bert_en_uncased_L-2_H-512_A-8", "small_bert/bert_en_uncased_L-2_H-768_A-12", "small_bert/bert_en_uncased_L-4_H-128_A-2", "small_bert/bert_en_uncased_L-4_H-256_A-4", "small_bert/bert_en_uncased_L-4_H-512_A-8", "small_bert/bert_en_uncased_L-4_H-768_A-12", "small_bert/bert_en_uncased_L-6_H-128_A-2", "small_bert/bert_en_uncased_L-6_H-256_A-4", "small_bert/bert_en_uncased_L-6_H-512_A-8", "small_bert/bert_en_uncased_L-6_H-768_A-12", "small_bert/bert_en_uncased_L-8_H-128_A-2", "small_bert/bert_en_uncased_L-8_H-256_A-4", "small_bert/bert_en_uncased_L-8_H-512_A-8", "small_bert/bert_en_uncased_L-8_H-768_A-12", "small_bert/bert_en_uncased_L-10_H-128_A-2", "small_bert/bert_en_uncased_L-10_H-256_A-4", "small_bert/bert_en_uncased_L-10_H-512_A-8", "small_bert/bert_en_uncased_L-10_H-768_A-12", "small_bert/bert_en_uncased_L-12_H-128_A-2", "small_bert/bert_en_uncased_L-12_H-256_A-4", "small_bert/bert_en_uncased_L-12_H-512_A-8", "small_bert/bert_en_uncased_L-12_H-768_A-12", "albert_en_base", "electra_small", "electra_base", "experts_pubmed", "experts_wiki_books", "talking-heads_base"]

    map_name_to_handle = {
        'bert_en_uncased_L-12_H-768_A-12':
            'https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/3',
        'bert_en_cased_L-12_H-768_A-12':
            'https://tfhub.dev/tensorflow/bert_en_cased_L-12_H-768_A-12/3',
        'bert_multi_cased_L-12_H-768_A-12':
            'https://tfhub.dev/tensorflow/bert_multi_cased_L-12_H-768_A-12/3',
        'small_bert/bert_en_uncased_L-2_H-128_A-2':
            'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-2_H-128_A-2/1',
        'small_bert/bert_en_uncased_L-2_H-256_A-4':
            'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-2_H-256_A-4/1',
        'small_bert/bert_en_uncased_L-2_H-512_A-8':
            'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-2_H-512_A-8/1',
        'small_bert/bert_en_uncased_L-2_H-768_A-12':
            'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-2_H-768_A-12/1',
        'small_bert/bert_en_uncased_L-4_H-128_A-2':
            'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-128_A-2/1',
        'small_bert/bert_en_uncased_L-4_H-256_A-4':
            'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-256_A-4/1',
        'small_bert/bert_en_uncased_L-4_H-512_A-8':
            'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-512_A-8/1',
        'small_bert/bert_en_uncased_L-4_H-768_A-12':
            'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-768_A-12/1',
        'small_bert/bert_en_uncased_L-6_H-128_A-2':
            'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-6_H-128_A-2/1',
        'small_bert/bert_en_uncased_L-6_H-256_A-4':
            'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-6_H-256_A-4/1',
        'small_bert/bert_en_uncased_L-6_H-512_A-8':
            'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-6_H-512_A-8/1',
        'small_bert/bert_en_uncased_L-6_H-768_A-12':
            'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-6_H-768_A-12/1',
        'small_bert/bert_en_uncased_L-8_H-128_A-2':
            'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-8_H-128_A-2/1',
        'small_bert/bert_en_uncased_L-8_H-256_A-4':
            'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-8_H-256_A-4/1',
        'small_bert/bert_en_uncased_L-8_H-512_A-8':
            'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-8_H-512_A-8/1',
        'small_bert/bert_en_uncased_L-8_H-768_A-12':
            'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-8_H-768_A-12/1',
        'small_bert/bert_en_uncased_L-10_H-128_A-2':
            'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-10_H-128_A-2/1',
        'small_bert/bert_en_uncased_L-10_H-256_A-4':
            'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-10_H-256_A-4/1',
        'small_bert/bert_en_uncased_L-10_H-512_A-8':
            'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-10_H-512_A-8/1',
        'small_bert/bert_en_uncased_L-10_H-768_A-12':
            'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-10_H-768_A-12/1',
        'small_bert/bert_en_uncased_L-12_H-128_A-2':
            'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-12_H-128_A-2/1',
        'small_bert/bert_en_uncased_L-12_H-256_A-4':
            'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-12_H-256_A-4/1',
        'small_bert/bert_en_uncased_L-12_H-512_A-8':
            'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-12_H-512_A-8/1',
        'small_bert/bert_en_uncased_L-12_H-768_A-12':
            'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-12_H-768_A-12/1',
        'albert_en_base':
            'https://tfhub.dev/tensorflow/albert_en_base/2',
        'electra_small':
            'https://tfhub.dev/google/electra_small/2',
        'electra_base':
            'https://tfhub.dev/google/electra_base/2',
        'experts_pubmed':
            'https://tfhub.dev/google/experts/bert/pubmed/2',
        'experts_wiki_books':
            'https://tfhub.dev/google/experts/bert/wiki_books/2',
        'talking-heads_base':
            'https://tfhub.dev/tensorflow/talkheads_ggelu_bert_en_base/1',
    }

    map_model_to_preprocess = {
        'bert_en_uncased_L-12_H-768_A-12':
            'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
        'bert_en_cased_L-12_H-768_A-12':
            'https://tfhub.dev/tensorflow/bert_en_cased_preprocess/3',
        'small_bert/bert_en_uncased_L-2_H-128_A-2':
            'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
        'small_bert/bert_en_uncased_L-2_H-256_A-4':
            'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
        'small_bert/bert_en_uncased_L-2_H-512_A-8':
            'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
        'small_bert/bert_en_uncased_L-2_H-768_A-12':
            'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
        'small_bert/bert_en_uncased_L-4_H-128_A-2':
            'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
        'small_bert/bert_en_uncased_L-4_H-256_A-4':
            'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
        'small_bert/bert_en_uncased_L-4_H-512_A-8':
            'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
        'small_bert/bert_en_uncased_L-4_H-768_A-12':
            'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
        'small_bert/bert_en_uncased_L-6_H-128_A-2':
            'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
        'small_bert/bert_en_uncased_L-6_H-256_A-4':
            'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
        'small_bert/bert_en_uncased_L-6_H-512_A-8':
            'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
        'small_bert/bert_en_uncased_L-6_H-768_A-12':
            'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
        'small_bert/bert_en_uncased_L-8_H-128_A-2':
            'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
        'small_bert/bert_en_uncased_L-8_H-256_A-4':
            'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
        'small_bert/bert_en_uncased_L-8_H-512_A-8':
            'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
        'small_bert/bert_en_uncased_L-8_H-768_A-12':
            'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
        'small_bert/bert_en_uncased_L-10_H-128_A-2':
            'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
        'small_bert/bert_en_uncased_L-10_H-256_A-4':
            'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
        'small_bert/bert_en_uncased_L-10_H-512_A-8':
            'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
        'small_bert/bert_en_uncased_L-10_H-768_A-12':
            'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
        'small_bert/bert_en_uncased_L-12_H-128_A-2':
            'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
        'small_bert/bert_en_uncased_L-12_H-256_A-4':
            'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
        'small_bert/bert_en_uncased_L-12_H-512_A-8':
            'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
        'small_bert/bert_en_uncased_L-12_H-768_A-12':
            'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
        'bert_multi_cased_L-12_H-768_A-12':
            'https://tfhub.dev/tensorflow/bert_multi_cased_preprocess/3',
        'albert_en_base':
            'https://tfhub.dev/tensorflow/albert_en_preprocess/3',
        'electra_small':
            'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
        'electra_base':
            'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
        'experts_pubmed':
            'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
        'experts_wiki_books':
            'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
        'talking-heads_base':
            'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    }

    tfhub_handle_encoder = map_name_to_handle[bert_model_name]
    tfhub_handle_preprocess = map_model_to_preprocess[bert_model_name]
    
    os.makedirs(vectorize_data_path, exist_ok = True)
    
    pickle.dump(tfhub_handle_encoder, open(f'{vectorize_data_path}/tfhub_handle_encoder.pkl', "wb"))
    pickle.dump(tfhub_handle_preprocess, open(f'{vectorize_data_path}/tfhub_handle_preprocess.pkl', "wb"))

In [111]:
vectorize_op = kfp.components.create_component_from_func(vectorize_data,
                                                       output_component_file='vectorize_data_component.yaml',
                                                       base_image="python:3.8",
                                                       packages_to_install=['tensorflow'])

In [112]:
# Build model

In [113]:
def build_model(epochs: int,
                vectorize_data_path: comp.InputPath(str),
                preprocess_data_path: comp.InputPath(str()),
                model_path: comp.OutputPath(str())):
    
    import os
    import tensorflow as tf
    import tensorflow_hub as hub
    import tensorflow_text as text
    import pickle

    from pathlib import Path
    from official.nlp import optimization  # to create AdamW optimizer
    
    # Load Datasets   
    train_ds = tf.data.Dataset.load(f'{preprocess_data_path}/raw_train')
    val_ds = tf.data.Dataset.load(f'{preprocess_data_path}/raw_val')
    test_ds = tf.data.Dataset.load(f'{preprocess_data_path}/raw_test') 
    
    tfhub_handle_encoder = pickle.load(open(f'{vectorize_data_path}/tfhub_handle_encoder.pkl', "rb"))
    tfhub_handle_preprocess = pickle.load(open(f'{vectorize_data_path}/tfhub_handle_preprocess.pkl', "rb"))
    
    def build_classifier_model():
        text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
        preprocessing_layer = hub.KerasLayer(tfhub_handle_preprocess, name='preprocessing')
        encoder_inputs = preprocessing_layer(text_input)
        encoder = hub.KerasLayer(tfhub_handle_encoder, trainable=True, name='BERT_encoder')
        outputs = encoder(encoder_inputs)
        net = outputs['pooled_output']
        net = tf.keras.layers.Dropout(0.1)(net)
        net = tf.keras.layers.Dense(1, activation=None, name='classifier')(net)
        
        return tf.keras.Model(text_input, net)
        
    
    model = build_classifier_model() 
    
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=3e-5, epsilon=1e-08, clipnorm=1.0), 
                  loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), 
                  metrics=[tf.keras.metrics.SparseCategoricalAccuracy('accuracy')])
    
    #creating the preprocess directory
    os.makedirs(model_path, exist_ok = True)
    
    model.save(f'{model_path}/model.h5')

In [114]:
build_model_op = kfp.components.create_component_from_func(build_model,
                                                           output_component_file='build_model_component.yaml',
                                                           base_image="python:3.8",
                                                           packages_to_install=['tensorflow', 'tensorflow-hub', 'tf-models-official', 'tensorflow-text', 'keras','pathlib'])

In [115]:
# Train model

In [116]:
def train_model(epochs: int, batch_size: int, es: bool,
                model_path: comp.InputPath(str()),
                preprocess_data_path: comp.InputPath(str()), 
                train_path: comp.OutputPath(str())):

    import os
    import pickle
    import keras
    import tensorflow as tf
    import tensorflow_hub as hub
    import tensorflow_text as text
    
    from tensorflow.keras.callbacks import EarlyStopping
    from official.nlp import optimization  # to create AdamW optimizer
    
    train_ds = tf.data.Dataset.load(f'{preprocess_data_path}/raw_train')
    val_ds = tf.data.Dataset.load(f'{preprocess_data_path}/raw_val')
    test_ds = tf.data.Dataset.load(f'{preprocess_data_path}/raw_test')    
    
    model = keras.models.load_model(f'{model_path}/model.h5', custom_objects={'KerasLayer':hub.KerasLayer})
    
    """
     ## Train the model
    """
    
    model.fit(train_ds, validation_data=val_ds, epochs=epochs, batch_size=batch_size)
    
    #creating the preprocess directory
    os.makedirs(train_path, exist_ok = True)
    
    #saving the model 
    model.save(f'/mnt/model.h5', include_optimizer=False)
    
    # visualization of the results
    score, acc = model.evaluate(test_ds)
    
    visualization = f"""
    <html>
        <body>
            Accuracy: "{acc}"
        </body>
    </html>
    """
    metadata = {
        'outputs': [{
            'storage': 'inline',
            'source': visualization,
            'type': 'web-app',
        }]
    }
    from collections import namedtuple
    output = namedtuple('VisualizationOutput', ['echo', 'mlpipeline_ui_metadata'])
    return output('Visualization', json.dumps(metadata))  

In [117]:
train_op = kfp.components.create_component_from_func(train_model,
                                                     output_component_file='train_model_component.yaml',
                                                     base_image="python:3.8",
                                                     packages_to_install=['tensorflow', 'tensorflow-hub', 'tf-models-official', 'tensorflow-text', 'keras'])

In [118]:
@dsl.pipeline(
   name='transformer pipeline',
   description='An example pipeline that performs for a sentiment model'
)
def transformer_pipeline(
                   epochs:int, 
                   batch_size:int, 
                   es:bool,
                   load_data_path: str,
                   preprocess_data_path: str,
                   vectorize_data_path:str,
                   embedding_data_path:str,
                   model_path:str,
                   train_path:str,
                  ):
    
    download_IMDB_container = download_IMDB_op()
    download_FP_container = download_FP_op()
    preprocess_container = preprocess_op(batch_size, download_IMDB_container.output, download_FP_container.output).after(download_IMDB_container).after(download_FP_container)
    vectorize_container = vectorize_op(preprocess_container.output)
    model_container = build_model_op(epochs, vectorize_container.output, preprocess_container.output)
    trained_container = train_op(epochs, batch_size, es, model_container.output, preprocess_container.output).add_pvolumes({"/mnt": dsl.PipelineVolume(pvc="model-volume")})

In [119]:
epochs = 5
batch_size = 32
es = True

load_data_path = "/mnt"
preprocess_data_path = "preprocess_data"
vectorize_data_path = "vectorize_data"
model_path = "model"
train_path = "train"

In [120]:
USERNAME = "user@example.com"
PASSWORD = "hsb1234#"
NAMESPACE = "kubeflow-user-example-com"
HOST = 'http://istio-ingressgateway.istio-system.svc.cluster.local:80'

session = requests.Session()
response = session.get(HOST)

headers = {
    "Content-Type": "application/x-www-form-urlencoded",
}

data = {"login": USERNAME, "password": PASSWORD}
session.post(response.url, headers=headers, data=data)
session_cookie = session.cookies.get_dict()["authservice_session"]

client = kfp.Client(
    host=f"{HOST}/pipeline",
    cookies=f"authservice_session={session_cookie}",
)

arguments = {"epochs":epochs,
             "batch_size":batch_size,
             "es":es,
             "vectorize_data_path":vectorize_data_path,
             "load_data_path":load_data_path,
             "preprocess_data_path":preprocess_data_path,
             "model_path":model_path,
             "train_path":train_path,
            }

client.create_run_from_pipeline_func(pipeline_func=transformer_pipeline, arguments=arguments)

RunPipelineResult(run_id=abbb228c-c8d7-461b-bfbc-d6470d112fad)